0. Packages & options (≃ “Importing Packages”)

In [1]:
the.seed <-42
set.seed(the.seed)

In [13]:
install.packages("Seurat")
install.packages("remotes")
remotes::install_github("mojaveazure/seurat-disk")

install.packages("hdf5r", type = "binary")
install.packages("SeuratDisk", type = "binary")
install.packages("SeuratDisk")
install.packages("SeuratDisk", type = "binary")


The downloaded binary packages are in
	/var/folders/k4/d0pwylgd1tl4mf2_hqnm6l400000gn/T//Rtmpwfkr2C/downloaded_packages

The downloaded binary packages are in
	/var/folders/k4/d0pwylgd1tl4mf2_hqnm6l400000gn/T//Rtmpwfkr2C/downloaded_packages


Using GitHub PAT from the git credential store.




digest       (0.6.37   -> 0.6.39  ) [CRAN]
listenv      (0.9.1    -> 0.10.0  ) [CRAN]
future       (1.67.0   -> 1.68.0  ) [CRAN]
spatstat.... (3.1-4    -> 3.1-5   ) [CRAN]
spatstat.... (3.4-2    -> 3.4-3   ) [CRAN]
spatstat.... (3.6-0    -> 3.6-1   ) [CRAN]
promises     (1.4.0    -> 1.5.0   ) [CRAN]
stringr      (1.5.2    -> 1.6.0   ) [CRAN]
S7           (0.2.0    -> 0.2.1   ) [CRAN]
RcppArmad... (15.0.2-2 -> 15.2.2-1) [CRAN]
reshape2     (1.4.4    -> 1.4.5   ) [CRAN]
ggplot2      (4.0.0    -> 4.0.1   ) [CRAN]
tinytex      (0.57     -> 0.58    ) [CRAN]
yaml         (2.3.10   -> 2.3.11  ) [CRAN]
purrr        (1.1.0    -> 1.2.0   ) [CRAN]
progressr    (0.17.0   -> 0.18.0  ) [CRAN]
uwot         (0.2.3    -> 0.2.4   ) [CRAN]
spatstat.... (3.5-3    -> 3.6-0   ) [CRAN]
reticulate   (1.44.0   -> 1.44.1  ) [CRAN]


Installing 19 packages: digest, listenv, future, spatstat.univar, spatstat.random, spatstat.geom, promises, stringr, S7, RcppArmadillo, reshape2, ggplot2, tinytex, yaml, purrr, progressr, uwot, spatstat.explore, reticulate




The downloaded binary packages are in
	/var/folders/k4/d0pwylgd1tl4mf2_hqnm6l400000gn/T//Rtmpwfkr2C/downloaded_packages
── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/private/var/folders/k4/d0pwylgd1tl4mf2_hqnm6l400000gn/T/Rtmpwfkr2C/remotes6dac7e8bd960/mojaveazure-seurat-disk-877d4e1/DESCRIPTION’ ... OK
* preparing ‘SeuratDisk’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘SeuratDisk_0.0.0.9021.tar.gz’


The downloaded binary packages are in
	/var/folders/k4/d0pwylgd1tl4mf2_hqnm6l400000gn/T//Rtmpwfkr2C/downloaded_packages


Warning message:
“package ‘SeuratDisk’ is not available as a binary package for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Warning message:
“package ‘SeuratDisk’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Warning message:
“package ‘SeuratDisk’ is not available as a binary package for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


In [15]:
## Packages 
library(Seurat)
library(SeuratDisk)   
library(Matrix)
library(dplyr)
library(ggplot2)
library(patchwork)


Warning message:
“package ‘ggplot2’ was built under R version 4.5.2”


ERROR: Error in value[[3L]](cond): Package ‘ggplot2’ version 4.0.0 cannot be unloaded:
 Error in unloadNamespace(package) : namespace ‘ggplot2’ is imported by ‘cowplot’, ‘ggridges’, ‘scattermore’, ‘ggrepel’, ‘Seurat’, ‘patchwork’, ‘plotly’, ‘sctransform’ so cannot be unloaded



1. Import des données (≃ adata = sc.read_h5ad(...))

In [18]:
## 1.1 Conversion h5ad -> h5seurat (à faire UNE FOIS) 

#elodie 
Convert("/Users/elodiehusson/Desktop/AD & PD/AD_PD_CTRL.h5ad", dest = "h5seurat", overwrite = TRUE)

#Yara
# Convert("C:/Users/yarad/Desktop/x/Masters/Master BMC - Sorbonne/M2/Single Cell/Project/Coding Project/AD_PD_CTRL.h5ad", dest = "h5seurat", overwrite = TRUE)


Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900

Adding X as data

Adding X as counts

Adding meta.features from var

Adding X_umap as cell embeddings for umap

Adding batch_condition to miscellaneous data

Adding citation to miscellaneous data

Adding class_colors to miscellaneous data

Adding genome to miscellaneous data

Adding organism to miscellaneous data

Adding organism_ontology_term_id to miscellaneous data

Adding schema_reference to miscellaneous data

Adding schema_version to miscellaneous data

Adding subclass_colors to miscellaneous data

Adding subtype_colors to miscellaneous data

Adding title to miscellaneous data

Adding uid to miscellaneous data



In [ ]:

## 1.2 Chargement dans un objet Seurat ----
sobj <- LoadH5Seurat("AD_PD_CTRL.h5seurat")

sobj
head(colnames(sobj))      # noms des cellules
head(rownames(sobj))      # noms des gènes
head(sobj@meta.data)      # équivalent à adata.obs




2. QC & métadonnées (≃ sc.pp.calculate_qc_metrics + plots)

En Python tu as :

sc.pp.calculate_qc_metrics(adata, inplace=True)

En R/Seurat, on utilise déjà nFeature_RNA, nCount_RNA et on ajoute le % mitochondrial :


In [ ]:

## 2.1 Ajout des métriques QC ----

# gènes mitochondriaux humains typiques
sobj[["percent.mt"]] <- PercentageFeatureSet(sobj, pattern = "^MT-")

# si besoin : ribosomal, hemoglobin, etc. comme dans le .Rmd
sobj[["percent.ribo"]] <- PercentageFeatureSet(sobj, pattern = "^RPL|^RPS")
sobj[["percent.hb"]]   <- PercentageFeatureSet(sobj, pattern = "^HB[AB]")

head(sobj@meta.data)




2.2 Visualisation des QC metrics


In [ ]:
(≃ sc.pl.violin(adata, ['n_counts','n_genes']) + scatter)

## Violin plots nFeature, nCount, percent.mt ----
VlnPlot(
  sobj,
  features = c("nFeature_RNA", "nCount_RNA", "percent.mt"),
  ncol = 3,
  pt.size = 0.1
)

## Scatter plots (équivalent des scatter QC dans scanpy) ----
FeatureScatter(sobj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA") +
FeatureScatter(sobj, feature1 = "nCount_RNA", feature2 = "percent.mt")




3. Filtrage des cellules (≃ “Filtering low quality cells / mt / ribo / hb”)


In [ ]:

## 3.1 Choix des thresholds (inspirés du .Rmd + ton notebook) ----
minGenes <- 200      # comme dans la prof
minUMI   <- 1000     # à adapter
maxMT    <- 15       # % de mitochondrie max (exemple)


In [ ]:

## 3.2 Création d'un objet filtré ----
sobj_filtrd <- subset(
  sobj,
  subset = nFeature_RNA > minGenes &
           nCount_RNA   > minUMI   &
           percent.mt   < maxMT
)

## Visualisation post-filtre ----
VlnPlot(
  sobj_filtrd,
  features = c("nFeature_RNA", "nCount_RNA", "percent.mt"),
  ncol = 3,
  pt.size = 0.1
)


In [ ]:

#Si tu veux exclure certains gènes (ribosomal, hémoglobine) comme mentionné dans ton notebook :

genes_to_remove <- grep("^RPL|^RPS|^HB[AB]", rownames(sobj_filtrd), value = TRUE)

sobj_filtrd <- sobj_filtrd[ setdiff(rownames(sobj_filtrd), genes_to_remove), ]


4. Normalisation (≃ sc.pp.normalize_total + sc.pp.log1p)




5. HVG – Highly Variable Genes (≃ sc.pp.highly_variable_genes)


In [ ]:

sobj_filtrd <- FindVariableFeatures(
  sobj_filtrd,
  selection.method = "vst",
  nfeatures = 3000
)


In [ ]:

# liste des HVG
hvg <- VariableFeatures(sobj_filtrd)
length(hvg)


6. Scaling & PCA (≃ sc.pp.scale + sc.tl.pca)


In [ ]:
sobj_filtrd <- ScaleData(sobj_filtrd, features = hvg)

sobj_filtrd <- RunPCA(
  sobj_filtrd,
  features = hvg,
  npcs = 50 # à adapter selon ton notebook
)

ElbowPlot(sobj_filtrd)  # pour choisir combien de PC garder


7. Voisins, clustering, UMAP


In [ ]:
#7.1 Graph des voisins

sobj_filtrd <- FindNeighbors(
  sobj_filtrd,
  dims = 1:30   # adapte au nb de PC retenues
)


In [ ]:

#7.2 Clustering Leiden ≃ Seurat FindClusters

sobj_filtrd <- FindClusters(
  sobj_filtrd,
  resolution = 0.5  # adapte à ce que tu as testé en Python
)

table(sobj_filtrd$seurat_clusters)


In [ ]:
#7.3 UMAP

sobj_filtrd <- RunUMAP(
  sobj_filtrd,
  dims = 1:30
)


In [ ]:

DimPlot(sobj_filtrd, reduction = "umap", group.by = "seurat_clusters", label = TRUE)
DimPlot(sobj_filtrd, reduction = "umap", group.by = "disease")
DimPlot(sobj_filtrd, reduction = "umap", group.by = "genetic_ancestry")
DimPlot(sobj_filtrd, reduction = "umap", group.by = "subtype")



8. Annotation manuelle (≃ section “Manual Annotation” + marker plots)


In [ ]:
## Exemple de gènes marqueurs (adaptés à ton dataset) ----
marker_genes <- c("CD3D", "CD4", "CD8A", "P2RY12", "GFAP")  # à adapter

# UMAP colored by expression
FeaturePlot(sobj_filtrd, features = marker_genes, cols = c("lightgrey", "red"))

# Violin plots par cluster
VlnPlot(sobj_filtrd, features = marker_genes, group.by = "seurat_clusters", pt.size = 0)

Ensuite tu peux renommer tes clusters (comme dans le .Rmd de la prof) :

new.cluster.ids <- c(
  "0" = "CD8_T",
  "1" = "CD4_T",
  "2" = "Microglia",
  "3" = "Astrocytes"
  # etc.
)

sobj_filtrd$celltype_manual <- plyr::mapvalues(
  x = sobj_filtrd$seurat_clusters,
  from = names(new.cluster.ids),
  to   = new.cluster.ids
)

DimPlot(sobj_filtrd, reduction = "umap", group.by = "celltype_manual", label = TRUE)




9. Annotation automatique (≃ CellTypist)


In [ ]:

# En Python tu utilises celltypist.
# En R, il n’y a pas CellTypist directement, mais tu peux utiliser par ex. SingleR ou scType.

# Pour rester simple (et puisque ce n’est pas dans le .Rmd prof), je te mets juste un squelette très court :

# Exemple *idée* avec SingleR (si tu veux pousser) :
# library(SingleR)
# library(celldex)
# ref <- HumanPrimaryCellAtlasData()
# sce <- as.SingleCellExperiment(sobj_filtrd)
# pred <- SingleR(test = sce, ref = ref, labels = ref$label.main)
# sobj_filtrd$celltype_auto <- pred$labels
# DimPlot(sobj_filtrd, group.by = "celltype_auto", label = TRUE)

# Tu peux aussi t’arrêter à l’annotation manuelle.



10. Differential Expression (≃ sc.tl.rank_genes_groups)


In [ ]:

## 10.1 Choisir l'identité par laquelle comparer ----
Idents(sobj_filtrd) <- "disease"

## 10.2 Comparer un groupe vs référence ----
markers_AD_vs_ctrl <- FindMarkers(
  sobj_filtrd,
  ident.1 = "dementia || Alzheimer disease",  # adapte au label exact
  ident.2 = "normal",
  logfc.threshold = 0.25,
  min.pct = 0.2,
  test.use = "wilcox"
)


In [ ]:

head(markers_AD_vs_ctrl)


In [ ]:

Idents(sobj_filtrd) <- "seurat_clusters"

markers_all <- FindAllMarkers(
  sobj_filtrd,
  only.pos = TRUE,
  min.pct = 0.25,
  logfc.threshold = 0.25
)

head(markers_all)



11. Pseudo-bulk (≃ adata.to_df().groupby(adata.obs["donor_id"]).sum())

In [ ]:


## 11.1 Récupérer la matrice de counts ----
counts <- GetAssayData(sobj_filtrd, slot = "counts")

## 11.2 Vérifier qu'on a bien une colonne donor_id dans la metadata ----
head(sobj_filtrd$donor_id)

## 11.3 Agrégation pseudo-bulk par donneur ----
donors <- sobj_filtrd$donor_id

pb_mat <- sapply(
  X = split(seq_len(ncol(counts)), donors),
  FUN = function(cells) Matrix::rowSums(counts[, cells, drop = FALSE])
)

pb_mat <- as.matrix(pb_mat)  # gènes x donneurs

dim(pb_mat)
colnames(pb_mat)  # donneurs

